In [1]:
import scipy.io as sio
import numpy as np
import os
import re
import struct
import math

In [2]:
## setup algoParams
gamma_Hz_per_Tesla = 42.577481e6;
###########.  possibly a dictionary for this .##################
species1_name = 'water';
species1_frequency = 4.70;
species1_relAmps = 1;
species2_name = 'fat';
species2_frequency = [0.90, 1.30, 1.60, 2.02, 2.24, 2.75, 4.20, 5.19, 5.29]; # 9-peak model
species2_relAmps   = [  88,  642,   58,   62,   58,    6,   39,   10,   37]; # Hamilton G, et al. NMR Biomed. 24(7):784-90, 2011. PMID: 21834002
#############################################################
decoupled_estimation = True; # flag for decoupled R2 estimation
Fibonacci_search = True; # flag for Fibonacci search
B0_smooth_in_stack_direction = False; # flag for B0 smooth in stack direction
multigrid = True; # flag for multi-level resolution `pyramid
estimate_R2 = True; # flag to estimate R2star
verbose = True;
process_in_3D = True; # flag to process in 3D (default True)
R2star_calibration = True; # flag to perform R2* calibration (default False)
ICM_iterations = 2; # ICM iterations
F = 100; # number of discretized B0 values
mu = 10; # regularization parameter
R2_stepsize = 1; # R2 stepsize in s^-1
max_R2 = 120; # maximum R2 in s^-1
max_label_change = 0.1; # 
fine_R2_stepsize = 1.0; # Fine stepsize for discretization of R2(*) [s^-1] (used in decoupled fine-tuning step)
coarse_R2_stepsize = 10.0; # Coarse stepsize for discretization of R2(*) [s^-1] (used for joint estimation step, value 0.0 --> Decoupled estimation only
water_R2 = 0.0; # Water R2 [sec-1]
fat_R2s = np.zeros((1,9)); # fat peak R2s [s^-1]
R2star_calibration_max = 800; # max R2* to use for calibration [s^-1] (default 800)
R2star_calibration_cdf_threshold = 0.98; ## threshold for R2* calibration cumulative density function [0,1] (default 0.98)

In [3]:
def CoilCombine(im1):
    #permute or switch dimesnions - coil dimension the fourth one and the TE the third
    im1 = np.transpose(im1, (0, 1, 4, 3, 2))
    
    [sx, sy, N, C] = np.shape(im1)
    filtsize = 7
    
    im2 = np.zeros(sx, sy, 0, 0, N)
    Rs = np.zeros(sx, sy, C, C)
    
    
    for kc1 in range(C):
        for kc2 in range(C):
            for kn in range(N):
                Rs[:,:,kc1,kc2] = Rs[:,:,kc1,kc2] + signal.convolve2d(np.ones(filtsize), np.multiply(im1[:,:,kn,kc1],np.conj(im1[:,:,kn,kc2])), mode = 'same')
                ##### np.multiply should work and if not, maybe try * because it also does vector multiplication in python
    for kx in range(sx):
          for ky in range(sy):
                [U,S] = np.linalg.svd(np.squeeze(Rs[kx,ky,:,:]));
                myfilt = U[:,0]; 
                #im2[kx,ky,0,0,:] = myfilt'*reshape(squeeze(im1[kx,ky,:,:]).',[C N]);<---????????
                
    if type(im1) == 'float32':
        im2 = np.array(im2, dtype= float32)

In [4]:
def CoilCombine3D(images):
    
    [sx,sy,sz,C,N] = np.shape(images)

    # Maintain the data type
    ims = np.zeros([sx,sy,sz,1,N], dtype = np.dtype(images))
    
    for kz in range(sz):
          im2[:,:,kz,0,:] = coilCombine(images[:,:,kz,:,:])

    

In [5]:
i = 0
path = '/Users/joseferpaz/Documents/Vanderbilt Internship/fw_i3cm1i_3pluspoint_berglund_QPBO/test_cases' #HARDCODED (needs to be replaced eventually)
file_list = os.listdir(path)

edit_path = path + '/' + file_list[i]
mat = sio.loadmat(edit_path)

### Loading the objects ###################################################################
image_imDataParams = mat['imDataParams']['images']
TE_imDataParams = mat['imDataParams']['TE']
FieldStrength_imDataParams = mat['imDataParams']['FieldStrength']
PrecessionIsClockwise_imDataParams = mat['imDataParams']['PrecessionIsClockwise']
Mask_imDataParams = mat['imDataParams']['mask']
Voxel_imDataParams = mat['imDataParams']['mask']



# TEDIOUS and varies for some reason #

#images
image_squeezed = np.squeeze(image_imDataParams)
image_imDataParams = image_imDataParams[0,0] #HARDCODED replace this as well.. somehow

#echo times (TE) #double check
test_TE1 = TE_imDataParams[0,0]
test_TE2 = np.squeeze(TE_imDataParams)
TE_imDataParams = TE_imDataParams[0,0]
TE_imDataParams = np.squeeze(TE_imDataParams)

#Field Strength
FieldStrength_imDataParams = np.squeeze(FieldStrength_imDataParams)

#PrecessionIsClockwise
PrecessionIsClockwise_2 = PrecessionIsClockwise_imDataParams[0,0]
PrecessionIsClockwise_imDataParams = np.squeeze(PrecessionIsClockwise_imDataParams)

#MASK
Mask_imDataParams = Mask_imDataParams[0,0]



        ##getting dimensions
print "IMAGES"
nx, ny, nz, nc, ne = np.shape(image_imDataParams)
print "The dimensions are:" , np.shape(image_imDataParams)
print "Regular indexing:", np.shape(image_imDataParams)
print "Squeeeeze:", np.shape(image_squeezed)
print '-' * 50

print "TE"
print TE_imDataParams
print "and the length  is:" , len(TE_imDataParams)
print 'but now lets break it down:'
print "indexing:", test_TE1
print "IF we index it to [0]:", test_TE1[0]
print "squeezing:", test_TE2
print "squeeze the squeeze one again:", np.squeeze(test_TE2)
print '-' * 50

print "FIELD STRENGTH"
#print "This is the field strength info:" , Field_strength
print "And here it is after the squeeze:",  FieldStrength_imDataParams
print '-' * 50

print "CLOCKWISE"
print PrecessionIsClockwise_imDataParams
print PrecessionIsClockwise_2

print "MASK"
#print np.shape(np.squeeze(Mask_imDataParams))
print "The mask param:", np.shape(Mask_imDataParams)
print "The three dimensions:", nx, ny, nz
print "They should be equal in size"

#print Field_strength[0]
#print Field_strength[0,0]

IMAGES
The dimensions are: (192, 192, 4, 1, 6)
Regular indexing: (192, 192, 4, 1, 6)
Squeeeeze: ()
--------------------------------------------------
TE
[ 0.001436  0.003084  0.004732  0.00638   0.008028  0.009676]
and the length  is: 6
but now lets break it down:
indexing: [[ 0.001436  0.003084  0.004732  0.00638   0.008028  0.009676]]
IF we index it to [0]: [ 0.001436  0.003084  0.004732  0.00638   0.008028  0.009676]
squeezing: [[ 0.001436  0.003084  0.004732  0.00638   0.008028  0.009676]]
squeeze the squeeze one again: [[ 0.001436  0.003084  0.004732  0.00638   0.008028  0.009676]]
--------------------------------------------------
FIELD STRENGTH
And here it is after the squeeze: [[3]]
--------------------------------------------------
CLOCKWISE
[[1]]
[[1]]
MASK
The mask param: (192, 192, 4)
The three dimensions: 192 192 4
They should be equal in size


In [6]:

validParams = 0

if 'verbose' not in locals():
    verbose = False
    print verbose
'''else:
    if (verbose == 1 or verbose == True):
        verbose = True
        
    else:
        verbose = False
        
    print verbose'''

print verbose

True


In [7]:

if not image_imDataParams.any(): #(just assign it to a variable called images and use it here)
    print 'Warning: input must contain field ''images'' with size [nx ny nz ncoils nTE]'
elif len(np.shape(image_imDataParams)) != 5:
    print 'Warning: imDataParams.images should have dimensions [nx ny nz ncoils nTE]' # also produces error for nTE=1 <-- Maybe not in python
else:
    [nx, ny, nz, ncoils, nTE] = np.shape(image_imDataParams)
    print nx, ny, nz, ncoils, nTE
    
#print not image_imDataParams.any()

192 192 4 1 6


In [8]:
if not TE_imDataParams.any():
    print 'Warning: input must contain field ''TE'' with number of elements nTE)'
elif len(TE_imDataParams) != nTE:
    print 'Warning: TE_imDataParams has length %d (expected %d)' % (len(TE_imDataParams), ne)
print "TE parameters has length %d (expected %d)" % (len(TE_imDataParams), ne)

TE parameters has length 6 (expected 6)


In [9]:
print type(FieldStrength_imDataParams.any())#why is this one not a boolean... ?
print type(verbose)

print type(image_imDataParams.any())
print type(TE_imDataParams.any())


#if FieldStrength_imDataParams not in locals():
 #   print "This doesn't exist and yes I treat is like a variable with a single element instead of an array..."
    
if 'FieldStrength_imDataParams' not in locals():
    FieldStrength_imDataParams = 1.5
    print "Warning: defaulting FieldStrength_imDataParams to 1.5"
else:
    print "It's in here."

<type 'numpy.ndarray'>
<type 'bool'>
<type 'numpy.bool_'>
<type 'numpy.bool_'>
It's in here.


In [10]:
if 'PrecessionIsClockwise_imDataParams' not in locals():
    PrecessionIsClockwise = 1
    print "Warning: Defaulting PrecessionIsClockwise_imDataParams to 1"
else:
    print "We good!"

We good!


In [11]:
if not Mask_imDataParams.any():
    Mask_imDataParams = np.ones(nx, ny, nz)
    print 'Warning: Defaulting Mask_imDataParams to true [%d, %d, %d]' % (nx, ny, nz)
else:
    print Mask_imDataParams.dtype
    Mask_imDataParams = np.array(Mask_imDataParams, dtype = np.bool) #Make sure this worked
    print Mask_imDataParams.dtype
    

uint8
bool


In [12]:
if nTE < 3:
    print "Warning: A minimum of 3 evenly spaced echoes are required, only %d echoes were provided." % nTE

In [13]:
if ncoils>1:
    image_imDataParams = coilCombine3D(image_imDataParams)
    ncoils = 1

In [14]:
if PrecessionIsClockwise_imDataParams != 1:
    image_imDataParams = np.conj(image_imDataParams)
    PrecessionIsClockwise_imDataParams = 1

In [15]:
TE_imDataParams = TE_imDataParams[:] # <-- UNECESSARY ?

sorted_idx = np.argsort(TE_imDataParams)
TE_imDataParams = TE_imDataParams[sorted_idx]
image_imDataParams = image_imDataParams[:,:,:,:,sorted_idx]
print TE_imDataParams

[ 0.001436  0.003084  0.004732  0.00638   0.008028  0.009676]


In [16]:
print nx, ny, nz 
#nTE = 5
TE_usec_unique = np.unique(np.around((10**6)*TE_imDataParams)) ##<------ just 0. ?
print 'TE_usec_unique is the unique rounded values of of the echoes ---->', TE_usec_unique
print '_'*100

nTE_usec_unique = len(TE_usec_unique)
print 'nTE_usec_unique is the number of unique echoes --->', nTE_usec_unique
print '_'*100

print 'nTE is set earlier in the script --->', nTE
print "Now if nTE doesn't match nTE_usec_unique, the redundant echoes are averaged..  "
print '-'*100

if nTE != nTE_usec_unique:
    if verbose:
        print "Warning: Found %d unique echo times out of %d provided echo times : Redundant echo times will be averaged"% (nTE_usec_unique, nTE)
        
    images_tmp = np.zeros((nx, ny, nz, 1, nTE_usec_unique)) 
    #print 'The dimensions of images_tmp(array of zeros) :', images_tmp
    
    TE_tmp = np.zeros((nTE_usec_unique,1))
    print 'The dimensions of TE_tmp(array of zeros) :', TE_tmp
    print "What does tmp stand for?"
    print 'The following loop runs 7 times.. is it supposed to?'
    

    for kTE in range(nTE_usec_unique):
        
        print '-' * 50
        print 'LOOP %d' % kTE ### IT just runs one time
        idx_TE = np.nonzero(TE_usec_unique[kTE] == TE_imDataParams[:]) ## ??????? why giving me 0... should it? it has to be wrong
        
        print idx_TE 
        print len(idx_TE[:])
        print np.sum(image_imDataParams[:,:,:,0,idx_TE], axis = 4)
        
        images_tmp[:,:,:,0,kTE] = np.sum(image_imDataParams[:,:,:,0,idx_TE], axis = 4)/len(idx_TE[:])#problem line... or is it?
        
        TE_tmp[kTE] = TE_imDataParams[idx_TE[0]]
        
        print TE_tmp[kTE]
        
        ########## DEBUGGING!!!!! #############
        """
        ### Just add along the axis
        number_test = np.sum(image_imDataParams[:,:,:,0,idx_TE], axis = 4)
       
        #### what the last dimension is
        print 'lenght', len(idx_TE)
        len_idx = len(idx_TE)
        
        
        ## Get info
        print len(image_imDataParams[:,:,:,0,idx_TE])
        
        print np.shape(image_imDataParams[:,:,:,0,idx_TE])
        
        n_t = np.sum(image_imDataParams[:,:,:,0,idx_TE], axis = 4)
        #n_t
        n_t = n_t/len_idx
        #print n_t
        
        
        print np.shape(n_t)
        print np.shape(images_tmp[:,:,:,:,kTE]) ###SUbstituted the last 0 for a : but maybe you need to change the other var
        ### with the one above you could fit the summed array into the other
        or maybe you could shape the sum to fit the images_tmp but you might be sacrificing data
        
    
        #images_tmp[:,:,:,0,kTE] = np.sum(image_imDataParams[:,:,:,0,idx_TE], axis = 4)/len(idx_TE[:]) ###CHECK THE SUM FUNCTION"""
        ########## DEBUGGING!!!!! #############
        
    images_imDataParams = images_tmp
    TE_imDataParams = TE_tmp
    
    #images_tmp, TE_tmp = None


192 192 4
TE_usec_unique is the unique rounded values of of the echoes ----> [ 1436.  3084.  4732.  6380.  8028.  9676.]
____________________________________________________________________________________________________
nTE_usec_unique is the number of unique echoes ---> 6
____________________________________________________________________________________________________
nTE is set earlier in the script ---> 6
Now if nTE doesn't match nTE_usec_unique, the redundant echoes are averaged..  
----------------------------------------------------------------------------------------------------


In [17]:
print TE_imDataParams

dTE_usec = np.around((10**6)*np.diff(TE_imDataParams)) ##CHECK  DIFF FUNCTION #loop is unecesary
dTE_candidates = dTE_usec#[:]

ldTE = len(dTE_usec)
print ne
print ne-1

print "The loop begins HERE:"
print "Why is there even a loop?"

########################## MAKING THINGS WORK ####################################################
dTE_candidates = np.cumsum(dTE_usec) #<------ By doing this we get the matlab output.

#ldTE = ldTE - 1
###################################################################################################
for idx_dTE in range(ldTE):
    #dTE_candidates = ([ dTE_candidates[idx_dTE:-1], np.cumsum(dTE_usec[idx_dTE:-1]) ]);
    #dTE_candidates_2 = np.cumsum(dTE_usec[idx_dTE:-1])#[idx_dTE:-1]
    
    dTE_candidates_n = (dTE_candidates, np.cumsum(dTE_usec[idx_dTE:-1]) ) #the unique thing seems unnecesary 
    #print "dTE_candidates_n:", dTE_candidates_n
    
    #dTE_candidates_1 = np.unique(dTE_candidates)#delete only one ) next to the #..., (np.cumsum(dTE_usec[idx_dTE:-1]))])
    #dTE_candidates_sum = np.cumsum(dTE_usec[idx_dTE:-1]) # get rid of the [idx_dTE:-1] to make it look like the matlab output
    #print '-'*20,'guinea pigs','-'*20
    #print "dTE_candidates_sum:", dTE_candidates_sum
    #print "dTE_candidates_1:", dTE_candidates_1
    #print '='*20,'End OF Loop','='*20
    
                    
    
#print '#'*20,'LOOP IS OVER','#'*20   
#print 'before:', dTE_candidates_n
#dTE_candidates_n2 = np.unique(dTE_candidates_n[0][1]);
#dTE_candidates_n = np.unique(dTE_candidates_n[0][0]);

#print 'after [0][0] :', dTE_candidates_n
#print 'after [0][1] :', dTE_candidates_n2

#print dTE_candidates_1
#print dTE_candidates_sum
#########################################################################################################

best_echoes = []

#initialize main Dictionary
dTE_groups = {}

for idx_dTE_candidate in range(len(dTE_candidates)):
    
    print "\n"
    print "ITERATION #: ", idx_dTE_candidate
    
    target_dTE_usec = dTE_candidates[idx_dTE_candidate]
    ne = len(TE_imDataParams)

    
    for idx in range(ne-1):
        
        print "sub - i #: ", idx
        
        # initialize the dictionary within the existing one to mimic the cell array
        dTE_groups[idx] = {}
        
        dTE_groups[idx]['dTE_usec'] = dTE_usec[idx]
        dTE_groups[idx]['echoes'] = [idx, idx+1] ### like matlab [idx+1, idx+2]
        
    
    
    ### THE fact that target and group are stuck to a number each might be a problem
    #print dTE_groups[idx]['echoes']
    for idx in range(ne-1):
        
        print 'target:', target_dTE_usec
        print 'group idx:', dTE_groups[idx]['dTE_usec']
        
        if (dTE_groups[idx]['dTE_usec'] == target_dTE_usec):## ALL OF THEM SHOULD BE EQUAL
            print "THEY ARE EQUAL"
            #pass
            
            
        ###CORNCER CASE
        
        elif (dTE_groups[idx]['dTE_usec'] < target_dTE_usec):
            print "TARGET IS BIGGER... combine with next group" 
            print 'is %d smalled than %d?' % (idx, (ne-2))
            
            if idx < (ne-2): 
                
                
                idx_plus1 = idx + 1
                #print dTE_groups[idx]['dTE_usec']
                #print dTE_groups[idx_plus1]['dTE_usec'] #it runs out of objects to call becasue the list ends because 
                
                dTE_groups[idx_plus1]['dTE_usec'] = dTE_groups[idx]['dTE_usec'] + dTE_groups[idx_plus1]['dTE_usec']
                dTE_groups[idx_plus1]['echoes'] = [dTE_groups[idx]['echoes'][0:-2] , dTE_groups[idx_plus1]['echoes'][1:-1]]
                
                print "to be printed on a list:"
                print dTE_groups[idx_plus1]['echoes'][1:-1]
                print dTE_groups[idx_plus1]['echoes'][0:-2]
                
                dTE_groups[idx]['dTE_usec'] = float('nan') #NaN
                dTE_groups[idx]['echoes'] = []
                
                print 'the weird sum thing', dTE_groups[idx_plus1]['dTE_usec']
                print 'the echoes list ', dTE_groups[idx_plus1]['echoes']
                print '-'*50
                print '\n'
                
            else:
                dTE_groups[idx]['dTE_usec'] = float('nan') #NaN
                dTE_groups[idx]['echoes'] = []
                
        else:
            #echo spacing too large
            dTE_groups[idx]['dTE_usec'] = float('nan') #NaN
            dTE_groups[idx]['echoes'] = []
    
        #Now we find a usable TE set
        echoes_to_use = []
        echoes_to_use_trial = []
        idx = 0

        a = len(echoes_to_use) > 0

        b = np.isnan(dTE_groups[idx]['dTE_usec'])

        d = dTE_groups[idx]['echoes']
        print 'd:',d

    
    
        while idx <= (ne - 1):
            #if (np.isnan(dTE_groups[idx]['dTE_usec']) and (len(dTE_groups[idx]['echoes']))>0):
            if a and b :
                print "THE LOOP HAS BEEN BROKEN and conditions were met"
                break

            elif not b:
                echoes_to_use_trial = [echoes_to_use_trial, d]
                echoes_to_use.append(idx)
                
                print 'echoes is now:'
                print echoes_to_use
                #print 'trial echoes:'
                #echoes_to_use_trial[idx] = dTE_groups[idx]['echoes']
                print echoes_to_use_trial
                #print echoes_to_use
                #print echoes_to_use_trial
            else:
                print "conditions are missing and none of the above were met"

            idx += 1
        
    echoes_to_use = np.sort(np.unique(echoes_to_use))
    
    if len(echoes_to_use) > len(best_echoes):
        best_echoes = echoes_to_use
            
        
                
                
        

  

[ 0.001436  0.003084  0.004732  0.00638   0.008028  0.009676]
6
5
The loop begins HERE:
Why is there even a loop?


ITERATION #:  0
sub - i #:  0
sub - i #:  1
sub - i #:  2
sub - i #:  3
sub - i #:  4
target: 1648.0
group idx: 1648.0
THEY ARE EQUAL
d: [0, 1]
echoes is now:
[0]
[[], [0, 1]]
echoes is now:
[0, 1]
[[[], [0, 1]], [0, 1]]
echoes is now:
[0, 1, 2]
[[[[], [0, 1]], [0, 1]], [0, 1]]
echoes is now:
[0, 1, 2, 3]
[[[[[], [0, 1]], [0, 1]], [0, 1]], [0, 1]]
echoes is now:
[0, 1, 2, 3, 4]
[[[[[[], [0, 1]], [0, 1]], [0, 1]], [0, 1]], [0, 1]]
echoes is now:
[0, 1, 2, 3, 4, 5]
[[[[[[[], [0, 1]], [0, 1]], [0, 1]], [0, 1]], [0, 1]], [0, 1]]
target: 1648.0
group idx: 1648.0
THEY ARE EQUAL
d: [0, 1]
echoes is now:
[0]
[[], [0, 1]]
echoes is now:
[0, 1]
[[[], [0, 1]], [0, 1]]
echoes is now:
[0, 1, 2]
[[[[], [0, 1]], [0, 1]], [0, 1]]
echoes is now:
[0, 1, 2, 3]
[[[[[], [0, 1]], [0, 1]], [0, 1]], [0, 1]]
echoes is now:
[0, 1, 2, 3, 4]
[[[[[[], [0, 1]], [0, 1]], [0, 1]], [0, 1]], [0, 1]]
echoe

In [18]:
if 'voxelSize_imDataParams' not in locals():
    voxelSize_imDataParams = [1, 1, 1]


In [19]:
if 'decoupled_estimation' not in locals():
    decoupled_estimation = True
        
if 'Fibonacci_search' not in locals():
    Fibonacci_search = True

if 'B0_smooth_in_stack_direction' not in locals():
    B0_smooth_in_stack_direction = False

if 'multigrid' not in locals():
    multigrid = True

if 'estimate_R2' not in locals():
    estimate_R2 = True
    
if 'verbose' not in locals():
    verbose = False

if 'process_in_3D' not in locals():
    process_in_3D = True

if 'R2star_calibration' not in locals():
    R2star_calibration = False
        
if 'species' not in locals():
    species1_name = 'water'
    species1_frequency = 4.70
    species1_relAmps = 1;
    species2_name = 'fat';
    species2_frequency = [0.90, 1.30, 1.60, 2.02, 2.24, 2.75, 4.20, 5.19, 5.29] # 9-peak model
    species2_relAmps   = [  88,  642,   58,   62,   58,    6,   39,   10,   37] # Hamilton G, et al. NMR Biomed. 24(7):784-90, 2011. PMID: 21834002

species1_relAmps = species1_relAmps / np.sum( species1_relAmps )
species2_relAmps = species2_relAmps / np.sum( species2_relAmps )

if 'ICM_iterations' not in locals():
        ICM_iterations = 2
        
if 'num_B0_labels' not in locals():
    num_B0_labels = 100

if 'mu' not in locals():
    mu = 10
    
if 'R2_stepsize' not in locals():
    R2_stepsize = 1
    
if 'max_R2' not in locals():
    max_R2 = 120
    
if 'fine_R2_stepsize' not in locals():
    fine_R2_stepsize = 0.1

if 'coarse_R2_stepsize' not in locals():
    coarse_R2_stepsize = 10.0


In [20]:
## the following part of the fw.... code seems a bit unecessary
## it seems as if in matlab whenever a variable is assigned a 1 or a 0, it treats the data as numeric reather than boolean
## but it appears as if though python does distinguish when it is being called to be used as boolean.
## I think

In [21]:

if 'water_R2' not in locals():
    algoParams.water_R2  = 0.0
    
    
if 'fat_R2s' not in locals():
    if (species2_name == 'water') or (species2_name == 'Water'):
        fat_R2s  = np.zeros(1,len(species1_frequency))
    else:
        fat_R2s  = np.zeros(1,len(species2_frequency))
    
if 'R2star_calibration_max' not in locals():
    R2star_calibration_max  = 800
    
    
if 'R2star_calibration_cdf_threshold' not in locals():
    R2star_calibration_cdf_threshold  = 0.98